# Road Graph Creation Workflow

This notebook creates a stored road graph file so that the Osmnx API is not called for every run and the same cleaning techniques are used for each road graph. This saves time that would otherwise be needed to rebuild graphs for each simulation run.

This notebook demonstrates code to run a wildfire evacuation agent based model. This notebook is a companion to the forthcoming manuscript, "Evaluating Routing Strategies for Emergency Evacuation: A Spatially Explicit Agent-Based Modeling Approach" by Rebecca Vandewalle, Furqan Baig, Santiago Nunez-Corrales, Jeon Young Kang, and Shaowen Wang.

Notebook author: Rebecca Vandewalle
<br>Last updated: 1-4-24

<a id='top'></a>
## Table of Contents
- [Initial setup](#setup)
- [Rebuild road graph](#rebuild)

<a id='setup'></a>
## Initial setup
([back to Table of Contents](#top))

These code cells import the primary code and set the parameters used for creating the road graphs.

In [1]:
# import functions from main code base
from FireABM_opt import *


In [2]:
# set road graph parameters - these were used in the accompanying manuscript
# this downloads road graphs for the city of Santa Rosa, California
central_addresses = ['100 Santa Rosa Ave, Santa Rosa, CA 95404']
names = ['Sta_Rosa_demo']
hhs = ["Santa_Rosa_tracts_hh.shp"]
fires = ["santa_rosa_fire.shp"]
distances = [2000, 8000]


<a id='rebuild'></a>
## Rebuild road graph
([back to Table of Contents](#top))

### CAUTION!!!

**Running the script in this notebook will overwrite contents of Sta_Rosa_2000.pkl and Sta_Rosa_8000.pkl**

Four road graph files are provided with this notebook; Sta_Rosa_2000.pkl, Sta_Rosa_8000.pkl, Sta_Rosa_2000_orig.pkl, and Sta_Rosa_8000_orig.pkl. Road network information are from OpenStreetMap, so road network data can be edited by OpenStreetMap users at any time. Sta_Rosa_2000.pkl and Sta_Rosa_8000.pkl are created using code in this notebook (Rebuild_Road_Graphs). If you run this notebook, you will overwrite the existing road network files with new ones. For replicability sake, Sta_Rosa_2000_orig.pkl and Sta_Rosa_8000_orig.pkl were the road network files used in the manuscript experiments. Because the base Open Street Map data has changed since the experiments were preformed, these files will be different than ones generated with the Rebuild_Road_Graphs code.

The graph building process can take a few minutes (but no more than ~5 minutes). Because of this, the next cell has a flag. If `run_rebuild` is set to `False`, the rebuilding graph code will not run. If you change the value of `run_rebuild` to `True` the rebuilding graph code will run.

In [3]:
# flag to rebuild road graphs
# set to True to run following cell
run_rebuild = False

The code in the next cell builds the road graphs and preprocesses the data in preparation for use in the experiments used in the manuscript. First it requests the raw files from OpenStreetMap, then fills in missing attributes, projects the graph, and finally adds functionality used in the simulation.

In [4]:
# rebuild road graphs
if run_rebuild:  # flag to protect this code block from auto running
    # iterate through parameters
    for indx in range(len(central_addresses)):
        for dist in distances:
            road_graph = ox.graph_from_address(
                central_addresses[indx], distance=dist, distance_type='bbox',
                network_type='drive',
                clean_periphery=True, truncate_by_edge=True)

            # adds bearings (directions)
            road_graph = ox.add_edge_bearings(road_graph)
            # projects graph
            road_graph = ox.project_graph(road_graph)
            # fills missing speeds, removes dead ends
            road_graph = cleanUp(road_graph)

            # adds unit speed
            road_graph = add_unit_speed(road_graph)
            # adds weights for road types
            road_graph = add_road_type_weights(road_graph)

            # exports nodes and edges for easier functionality
            gdf_nodes, gdf_edges = get_node_edge_gdf(road_graph)
            # creates bounding boxes
            (bbox, lbbox, poly, x, y) = create_bboxes(gdf_nodes, 0.01)

            # loads households shapefile
            hh_tract = load_shpfile(road_graph, ("households", hhs[indx]))
            # adds households to road graph
            road_graph, overlap = add_households(road_graph, hh_tract, hhs[indx],
                                                 "Tot_Est_HH", num_col=4,
                                                 bbox_poly=poly)

            # loads fire shapefile
            fire_file = load_shpfile(road_graph, ("fire_input", fires[indx]))
            # sets fire shapefile start time
            init_fire = fire_file[fire_file['SimTime'] == 60]
            # adds distance from fire
            road_graph = add_fire_distance(road_graph, init_fire)

            # exports nodes and edges again to add household and fire info
            gdf_nodes, gdf_edges = get_node_edge_gdf(road_graph)

            # normalizes certain attributes
            road_graph = normalize_edge_attribute(road_graph, 'length',
                                                  min(gdf_edges.length),
                                                  max(gdf_edges.length),
                                                  'length_n')
            road_graph = normalize_edge_attribute(road_graph, 'rt_weighted_len',
                                                  min(gdf_edges.rt_weighted_len),
                                                  max(gdf_edges.rt_weighted_len),
                                                  'rt_wght_len_n')
            road_graph = normalize_edge_attribute(road_graph, 'fire_dist',
                                                  min(gdf_edges.fire_dist),
                                                  max(gdf_edges.fire_dist),
                                                  'fire_dist_n')
            road_graph = invert_norm_edge_attribute(road_graph,
                                                    'fire_dist_n',
                                                    'inv_fire_dist_n')

            # exports graphs to file
            outname = names[indx]+'_'+str(dist)+'.pkl'
            nx.write_gpickle(road_graph, path=outname)
            print(outname)
